# Immoweb data analysis 

### Importing libraries

In [11]:
# Import H2O and Initialize
import h2o
from h2o.automl import H2OAutoML
from h2o.frame import H2OFrame
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Initialize H2O
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,4 mins 37 secs
H2O_cluster_timezone:,Europe/Brussels
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,6 days
H2O_cluster_name:,H2O_from_python_daryc_vrhyhf
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.886 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


### Load and replace 'missing' with 'missing_info' and NaN or empty cells with -1

In [12]:
# Load the cleaned data into a DataFrame
cleaned_data_path = 'data/data_20240313_cleaned.csv'
cleaned_df = pd.read_csv(cleaned_data_path, decimal=',')

# Fill empty cells with -1
cleaned_df.fillna(-1, inplace=True)

# Replace other missing value indicators
cleaned_df.replace(['Missing', 'MISSING'], 'missing_info', inplace=True)

# Save the modified DataFrame to a CSV file
modified_data_path = 'data/data_20240313_modified.csv'
cleaned_df.to_csv(modified_data_path, index=False)
print(f"Modified data saved to: {modified_data_path}")

Modified data saved to: data/data_20240313_modified.csv


### Convert to pandas DataFrame for outlier removal

In [ ]:
# Convert modified DataFrame to an H2OFrame
df_cleaned = H2OFrame(cleaned_df)

# Define the features to check for outliers
features_to_check = ['price', 'surface_land_sqm', 'total_area_sqm', 'nbr_bedrooms']

# Define the function to calculate IQR bounds within H2OFrame context
def calculate_bounds_h2o(data, feature):
    Q1 = data[feature].quantile(0.25).as_data_frame().iloc[0,0]
    Q3 = data[feature].quantile(0.75).as_data_frame().iloc[0,0]
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return lower_bound, upper_bound

# Apply the outlier removal process for each specified feature within the H2O context
for feature in features_to_check:
    lower_bound, upper_bound = calculate_bounds_h2o(df_cleaned, feature)
    df_cleaned = df_cleaned[(df_cleaned[feature] >= lower_bound) & (df_cleaned[feature] <= upper_bound)]


### Split the data into training and testing sets

In [18]:
# Split the cleaned data into training and testing sets
train, test = df_cleaned.split_frame(ratios=[.8], seed=42)

# Specify Target and Predictor Variables 'price' is the target variable
target = 'price'
predictors = df_cleaned.columns
predictors.remove(target)

# Ensure you use `train` for AutoML, which now contains the cleaned data
aml = H2OAutoML(max_models=20, seed=42, max_runtime_secs=600)
aml.train(x=predictors, y=target, training_frame=train)

# View the AutoML Leaderboard
lb = aml.leaderboard
print(lb.head(rows=lb.nrows))  # Print all rows instead of default (10 rows)

# Make Predictions
predictions = aml.leader.predict(test)
print(predictions.head())

AutoML progress: |█
09:26:00.738: AutoML: XGBoost is not available; skipping it.

██████████████████████████████████████████████████████████████| (done) 100%
model_id                                                 rmse          mse       mae       rmsle    mean_residual_deviance
GBM_4_AutoML_4_20240320_92600                         56071.4  3.144e+09     38093.1    0.178201               3.144e+09
GBM_grid_1_AutoML_4_20240320_92600_model_5            56586.1  3.20199e+09   38503.4    0.180184               3.20199e+09
GBM_3_AutoML_4_20240320_92600                         56641.1  3.20821e+09   38878      0.180007               3.20821e+09
GBM_2_AutoML_4_20240320_92600                         56881.9  3.23555e+09   39235.3    0.181088               3.23555e+09
GBM_1_AutoML_4_20240320_92600                         57167.3  3.2681e+09    39288.2    0.182329               3.2681e+09
GBM_5_AutoML_4_20240320_92600                         57611.8  3.31912e+09   40114.1    0.183589           

### Saving the AutoML Leader Model, save predictions 

In [19]:
# Save the model
model_path = h2o.save_model(model=aml.leader, path="my_model_path", force=True)
print(f"Model saved to: {model_path}")

# Save predictions to a CSV file
predictions_path = "predictions_AutoML.csv"
h2o.export_file(predictions, path=predictions_path, force=True)
print(f"Predictions saved to: {predictions_path}")

Model saved to: D:\Github\Projects\immo-eliza-ml\my_model_path\GBM_4_AutoML_4_20240320_92600


### Loading the Model and Making Predictions

In [21]:
# Load the saved model
try:
    loaded_model = h2o.load_model(path=model_path)
    # Assuming you have a dataset to predict
    predictions = loaded_model.predict(test)
    print(predictions.head())
except Exception as e:
    print("An error occurred while loading the model or making predictions:", e)



gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
  predict
   131388
   400777
   427198
   340823
   375817
   146890
   187533
   358309
   185071
   313738
[10 rows x 1 column]



### Retrieve the Model

In [22]:
predictions_df = pd.read_csv("predictions_AutoML.csv")

# Examine the first few rows
print(predictions_df.head())

         predict
0  131388.051656
1  400776.737004
2  427197.953980
3  340822.525499
4  375816.987828


### Calculate Evaluation Metrics

In [23]:
# Calculate evaluation metrics
actuals = test[target].as_data_frame().values.flatten()  

mse = mean_squared_error(actuals, predictions_df['predict'])
rmse = mean_squared_error(actuals, predictions_df['predict'], squared=False)
mae = mean_absolute_error(actuals, predictions_df['predict'])
r2 = r2_score(actuals, predictions_df['predict'])

# Print the metrics
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'R-squared (R2): {r2}')

d:\Github\Projects\immo-eliza-ml\venv\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install datatable (for Python 3.9 or lower), or polars and pyarrow (for Python 3.10 or above) and activate it using:

with h2o.utils.threading.local_context(polars_enabled=True, datatable_enabled=True):
    pandas_df = h2o_df.as_data_frame()

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


Mean Squared Error (MSE): 2952608966.817194
Root Mean Squared Error (RMSE): 54337.914634417044
Mean Absolute Error (MAE): 37062.96518401191
R-squared (R2): 0.8212804436379184


d:\Github\Projects\immo-eliza-ml\venv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
